In [ ]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)



In [ ]:
client.search_experiments()

In [ ]:
client.create_experiment(name="my-test-experiment")

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="tags.model = 'xgboost' and attributes.start_time > 1759415435000",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
    )

runs


In [ ]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")



In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [ ]:
run_id = "debe60ddd4ce4936a7ca70d2393c316e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-xgboost")

In [ ]:
client.search_registered_models()

In [ ]:
model_name = "nyc-taxi-xgboost"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

In [ ]:
model_version = 2
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)



In [ ]:
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}",
)


In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error


def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df["PU_DO"] = df["PULocationID"] + "_" + df["DOLocationID"]
    categorical = ["PU_DO"]
    numerical = ["trip_distance"]
    train_dicts = df[categorical + numerical].to_dict(orient="records")
    return dv.transform(train_dicts)

def test_model(stage, X_test, y_test, name="nyc-taxi-xgboost"):
    model = mlflow.pyfunc.load_model(f"models/{name}/{stage}")
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)
    return rmse


def test_model_run(run_id, X_test, y_test, name="nyc-taxi-xgboost"):
    # model = mlflow.pyfunc.load_model(f"models/{name}/{stage}")

    model = mlflow.pyfunc.load_model(f"runs:/{run_id}/models_mlflow")
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)
    return rmse




In [ ]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

df.head

In [ ]:
client.download_artifacts(
    run_id="6c8814541e8e4792940c0ea284e7a2dc",
    path="preprocessor",
    dst_path=".",
)

In [ ]:
# client.download_artifacts(
#     run_id="6c8814541e8e4792940c0ea284e7a2dc",
#     path="model",
#     dst_path="models",
# )


In [ ]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values




In [ ]:
run_id = "4ab881b1da5a4546ac041b25dbeca240"
%time test_model_run(name=model_name, run_id=run_id, X_test=X_test, y_test=y_test)

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)
